In [1]:
import tensorflow as tf

In [2]:
# The following functions can be used to convert a value to a type compatible
# with tf.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
def image_example(image_string, label):
#   image_shape = tf.image.decode_jpeg(image_string).shape

  feature = {
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

In [4]:
# Write the raw image files to `images.tfrecords`.
# First, process the two images into `tf.Example` messages.
# Then, write to a `.tfrecords` file.
base_path = "../leapGestRecog/"
people = ["00", "01", "02", "03", "04", "05", "06", "07", "08", "09"]
gestures = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10"]
gestures_map = {
    1: "palm", 
    2: "l", 
    3: "fist", 
    4: "fist_moved",
    5: "thumb", 
    6: "index", 
    7: "ok", 
    8: "palm_moved", 
    9: "c",
    10: "down"
}

record_file = '../images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
    for person in range(10):
        for label in range(1, 11):
            path = "%s/%s_%s/" % (str(person).zfill(2), str(label).zfill(2), gestures_map[label])
            for id in range(1, 201):
                filename = "frame_%s_%s_%s.png" % (str(person).zfill(2), str(label).zfill(2), str(id).zfill(4))
                # print(base_path + path + filename)
                image = tf.io.read_file(base_path + path + filename)
                image = tf.image.decode_png(image, channels=1)
                # image_string = cv2.imread(base_path + path + filename).tobytes()
                image_string = image.numpy().tobytes()
                tf_example = image_example(image_string, label)
                writer.write(tf_example.SerializeToString())